In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import csv
import os
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
directory = r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES OP\SEPT'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
cinetpay = pd.read_csv(latest_file_path,sep=';')
cinetpay.sample()



directory = r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES PMT\SEPT'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
pmt = pd.read_csv(latest_file_path)
pmt.sample()

,created_at,payment_date,operator,merchant_name,transaction_id,merchant_transaction_id,id_operator,external_transaction_id,phone_number,client_email,amount,fee_amount,merchant_amount,operation_origin,currency,statut,country,provider_id,provider_name
9497,2024-09-18 15:08:41,NaN,TMONEY_TG,1XBET TG,b269c1f8-333e-4179-852b-a0352cd6d808,2767715667,EA240918.150846.R362826,NaN,22893153450,NaN,50000.00,2500.00,52500.00,transfer,XOF,SUCCESS,TG,EA240918.150846.R362826@@@@@b269c1f8-333e-4179...,CinetPay_Payout_TMONEY_TG


In [4]:
#cinetpay = pd.read_csv(r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES OP\JUILLET\20240715_cinetpay_transfer_20240708_20240714.csv', sep=';')

In [5]:
#pmt = pd.read_csv(r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES PMT\JUILLET\E20240714_PMT_Cinetpay_20240708_20240714.csv', on_bad_lines='skip')

In [6]:
#SAUVEGARDER UNE COPIE DE LA DATAFRAME
dfop1=cinetpay.copy()
dfpmt1=pmt.copy()

In [7]:
cinetpay1=cinetpay[['Date Création (GMT)',
            'Date Modification',
            'Date de traitement',
           'ID Marchand',
           'ID Operateur',
           'Téléphone',
            'Opérateur',
           'Montant Envoyé',
           'Statut',
           'Commentaire'
]]

In [8]:
dfpmt = pmt.rename(columns={
        'created_at': 'Created Date',
        'payment_date': 'Payment Date',
        'operator': 'Operator',
        'merchant_name': 'Merchant Name',
        'transaction_id': 'Transaction ID',
        'id_operator': 'ID Opérateur',
        'phone_number': 'Phone Number',
        'amount': 'Montant',
        'fee_amount': 'Fee amount',
        'merchant_amount': 'Merchant amount',
        'statut': 'Statut'
    })

In [9]:
dfpmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16485 entries, 0 to 16484
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Created Date             16485 non-null  object 
 1   Payment Date             0 non-null      float64
 2   Operator                 16485 non-null  object 
 3   Merchant Name            16485 non-null  object 
 4   Transaction ID           16485 non-null  object 
 5   merchant_transaction_id  16485 non-null  object 
 6   ID Opérateur             16330 non-null  object 
 7   external_transaction_id  0 non-null      float64
 8   Phone Number             16485 non-null  int64  
 9   client_email             0 non-null      float64
 10  Montant                  16485 non-null  float64
 11  Fee amount               16485 non-null  float64
 12  Merchant amount          16485 non-null  float64
 13  operation_origin         16485 non-null  object 
 14  currency              

In [10]:
#EXTRAITRE LA DATE DANS UNE NOUVELLE COLLONNE
dfpmt['DateCourte'] = dfpmt['Created Date'].apply(lambda x: x.split(' ')[0])
cinetpay['DateCourte'] = cinetpay1['Date Création (GMT)'].apply(lambda x: x.split(' ')[0])

In [11]:
cinetpay.Statut.unique()

array(['VAL', 'REC', 'REJ', 'NEW'], dtype=object)

In [12]:
new_rec= cinetpay.loc[(cinetpay['Statut'] == 'NEW') |(cinetpay['Statut'] == 'REC') ]

In [13]:
rej= cinetpay.loc[(cinetpay['Statut'] == 'REJ')]

In [14]:
#MISE EN PLACE DE RECHERCHE X POUR RECUPERATION CHEZ L'OP
        # Vérification des correspondances entre A1 et B1
correspondance_statut_op= cinetpay.set_index('ID Marchand')['Statut']
correspondance_date_op = cinetpay.set_index('ID Marchand')['DateCourte']
correspondance_op = cinetpay.set_index('ID Marchand')['Opérateur']
correspondance_idoperator = cinetpay.set_index('ID Marchand')['ID Operateur']


dfpmt['DATEOP'] = dfpmt['Transaction ID'].map(correspondance_date_op)
dfpmt['STATUTOP'] = dfpmt['Transaction ID'].map(correspondance_statut_op)
dfpmt['OPERATOROP'] = dfpmt['Transaction ID'].map(correspondance_op)
dfpmt['IDOPERATOR'] = dfpmt['Transaction ID'].map(correspondance_idoperator)

In [15]:
#CONVERTIR EN FLOAT
#dfpmt['Montant'] = dfpmt['Montant'].str.replace(',', '.').astype(float)
#dfpmt['Fee amount'] = dfpmt['Fee amount'].str.replace(',', '.').astype(float)
#dfpmt['Merchant amount'] = dfpmt['Merchant amount'].str.replace(',', '.').astype(float)

In [16]:
# Définir les taux de commission pour chaque opérateur
commission_rates = {
        'AIRTELCD' : 0.018,
        'MPESACD': 0.028,
        'MOOVBF': 0.01,
        'OMCD':	0.01,
        'OMML':	0.008,
        'MOOVML': 0.008,
        'OMBF':	0.01,
        'OM': 0.008,
        'FLOOZ': 0.007,
        'FLOOZTG': 0.0080,
        'TMONEYTG': 0.0080,
        'MOMO':	0.0050,
        'MLTEL': 0.0080

}

# Fonction pour calculer les frais d'opérateur
def calculate_frais_op(row):
    operator = row['OPERATOROP']
    montant = row['Montant']
    commission_rate = commission_rates.get(operator, 0)
    return montant * commission_rate

# Ajouter une colonne 'frais_op' avec les frais d'opérateur calculés
dfpmt['frais_op'] = dfpmt.apply(calculate_frais_op, axis=1)

In [17]:
dfpmt['Frais_pmt'] = dfpmt['Fee amount'] - dfpmt['frais_op']

In [18]:
dfop= cinetpay.loc[(cinetpay['Statut'] == 'VAL')]

In [19]:
#NBSI PMT &CINETPAY
dfpmt['CINETPAY'] = dfpmt['Transaction ID'].isin(dfop['ID Marchand']).astype(int)
dfop['PMT'] = dfop['ID Marchand'].isin(dfpmt['Transaction ID']).astype(int)
dfpmt['NEW_REC'] = dfpmt['Transaction ID'].isin(new_rec['ID Marchand']).astype(int)
dfpmt['REJ'] = dfpmt['Transaction ID'].isin(rej['ID Marchand']).astype(int)

In [20]:
dfpmt['Nombre']= dfpmt['Montant']
dfop['Nombre']=dfop['Montant Envoyé']

In [21]:
#Afficher toutes les lignes
pd.reset_option('display.max_rows')

df_filteredpmt = dfpmt[dfpmt['CINETPAY'] == 1]

# Création du tableau croisé dynamique
tcdpmt = pd.pivot_table(
    df_filteredpmt,
    values=['Montant', 'Nombre','frais_op', 'Frais_pmt'],
    index=['DateCourte', 'OPERATOROP','Statut'],
    aggfunc={'Nombre': 'count','Montant': 'sum' ,'frais_op': 'sum', 'Frais_pmt': 'sum' },
    fill_value=0,
    margins=True,
    margins_name='Total')


df_filtered = dfop[(dfop['PMT'] == 1) | (dfop['PMT'] == 0)]

# Création du tableau croisé dynamique
tcdcinetpay = pd.pivot_table(
    df_filtered,
    values=['Montant Envoyé', 'Nombre'],
    index=['DateCourte', 'Opérateur','Statut'],
    aggfunc={'Nombre': 'count','Montant Envoyé': 'sum' },
    fill_value=0,
    margins=True,
    margins_name='Total'
)

In [22]:
tcdcinetpay

Montant Envoyé  Nombre
DateCourte Opérateur Statut                        
2024-09-18 AIRTELCD  VAL           12377690     287
           FLOOZ     VAL            2397000     350
           FLOOZTG   VAL            3511700     460
           MLTEL     VAL             234100      29
           MOMO      VAL              87800      25
           MOOVBF    VAL           12090500     685
           MPESACD   VAL           13998985     230
           OM        VAL           15245975     984
           OMBF      VAL           21480365    1259
           OMCD      VAL            4872190     134
           OMML      VAL            5167915     348
           TMONEYTG  VAL           11749310    1061
Total                             103213530    5852

In [23]:
tcdpmt

Frais_pmt      Montant  Nombre   frais_op
DateCourte OPERATOROP Statut                                            
2024-09-18 AIRTELCD   PENDING      64.00      2000.00       1      36.00
                      SUCCESS  365179.18  12375690.00     286  222762.42
           FLOOZ      SUCCESS   61680.49   2397008.00     350   16779.06
           FLOOZTG    SUCCESS  138620.78   3511702.00     460   28093.62
           MLTEL      SUCCESS    5150.20    234100.00      29    1872.80
           MOMO       SUCCESS    3226.75     87800.00      25     439.00
           MOOVBF     SUCCESS  576041.33  12090520.00     685  120905.20
           MPESACD    SUCCESS  134524.25  13998992.00     230  391971.78
           OM         SUCCESS  363447.72  15246030.00     984  121968.24
           OMBF       PENDING      30.00      1500.00       1      15.00
                      SUCCESS  940947.18  21478884.00    1258  214788.84
           OMCD       SUCCESS  184454.60   4872190.00     134   48721.90
           OMML       SUCCESS  113694.13   5167915.00     348   41343.32
           TMONEYTG   SUCCESS  461413.37  11749336.00    1061   93994.69
Total                         3348473.99 103213667.00    5852 1303691.86

In [24]:
maj = dfpmt.loc[(dfpmt['Statut'] == 'PENDING') & (dfpmt['CINETPAY'] == 1)]
maj

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre
5908,2024-09-18 04:23:43,NaN,AIRTEL_CD,1XBET CD,6137153a-be38-467c-b188-bd786493469d,2765922951,NaN,NaN,243972226477,NaN,...,2024-09-18,VAL,AIRTELCD,CI240918.0524.A48352,36.00,64.00,1,0,0,2000.00
15180,2024-09-18 22:24:16,NaN,OM_BF,Mel Bet BF,13edbff9-da37-4e1d-b872-8c0b918e98ef,2769013173,NaN,NaN,22657293300,NaN,...,2024-09-18,VAL,OMBF,CI240918.2224.86733693,15.00,30.00,1,0,0,1500.00


In [25]:
perte = dfpmt.loc[(dfpmt['Statut'] == 'FAILED') & (dfpmt['CINETPAY'] == 1)]
perte

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


In [26]:
trx_succes_a_rechercher = dfop.loc[(dfop['PMT'] == 0)]
trx_succes_a_rechercher

,Date Création (GMT),Date Modification,Date de traitement,ID CinetPay,ID Marchand,ID Operateur,Téléphone,Opérateur,Montant Envoyé,Marchand verifié ?,Statut,Commentaire,DateCourte,PMT,Nombre


In [27]:
result_trx_succes_a_rech = dfpmt.loc[(dfpmt['Transaction ID'] == '7606e7a9-9532-4c89-a004-6177f942362b')]
result_trx_succes_a_rech

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


In [28]:
trx_pending_abs_cinet = dfpmt.loc[(dfpmt['Statut'] == 'PENDING') & (dfpmt['CINETPAY'] == 0)]
trx_pending_abs_cinet

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre
5676,2024-09-18 13:27:54,NaN,AIRTEL_CD,OTRADA HOSPITALITY SARL RDC,c48fadff-6ed4-438d-ab0d-7107bd42b0e4,2767367403,NaN,NaN,243972256104,NaN,...,NaN,NaN,NaN,NaN,0.00,500.00,0,0,0,10000.00
5678,2024-09-18 13:28:03,NaN,OM_CD,OTRADA HOSPITALITY SARL RDC,eee78c67-f37e-41bf-a5fe-fb4577e424cb,2767354673,NaN,NaN,243840242228,NaN,...,NaN,NaN,NaN,NaN,0.00,150.00,0,0,0,3000.00
5944,2024-09-18 13:35:48,NaN,MPESA_CD,Betwinner CD,4e13ef70-ac79-4153-9eca-ff80f1801a02,138694223,NaN,NaN,243836005374,NaN,...,NaN,NaN,NaN,NaN,0.00,2800.00,0,0,0,70000.00
5997,2024-09-18 04:29:13,NaN,MPESA_CD,Betwinner CD,9ea25494-e16f-4203-891d-074cb2cb502b,138589052,EA240918.042919.K435766,NaN,243975505393,NaN,...,2024-09-18,REC,AIRTELCD,NaN,360.00,440.00,0,1,0,20000.00
6999,2024-09-18 13:43:15,NaN,AIRTEL_CD,OTRADA HOSPITALITY SARL RDC,4fd939e2-c40b-40ff-94ca-36fc6fc1bd2b,2767435371,NaN,NaN,243992835134,NaN,...,NaN,NaN,NaN,NaN,0.00,2000.00,0,0,0,40000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16436,2024-09-19 02:44:13,NaN,OM_BF,1XBET BF,bc9cff06-4a19-45e6-a3eb-9d6dc7ff9001,2769428421,EA240919.024419.K125348,NaN,22654464628,NaN,...,NaN,NaN,NaN,NaN,0.00,210.00,0,0,0,3500.00
16440,2024-09-18 21:21:59,NaN,FLOOZ,AMCDRC LTD CI,7964b3ee-6d7c-4c89-8589-babe0c9f6d03,138831842,EA240918.212209.J626410,NaN,2250172162563,NaN,...,2024-09-18,REC,FLOOZ,NaN,7.00,48.00,0,1,0,1000.00
16454,2024-09-19 02:45:22,NaN,OM_BF,Bet Winner BF,b016a56a-ea5f-4e8d-a32f-19e92cf9ff00,138873263,EA240919.024529.O308721,NaN,22655465836,NaN,...,NaN,NaN,NaN,NaN,0.00,3905.00,0,0,0,71000.00
16462,2024-09-19 02:46:23,NaN,OM_BF,1XBET BF,945405ba-1d7a-4f5d-b20c-687b0260d56b,2769431137,EA240919.024630.Y726990,NaN,22666801810,NaN,...,NaN,NaN,NaN,NaN,0.00,300.00,0,0,0,5000.00


In [29]:
trx_succes_abs_cinet = dfpmt.loc[(dfpmt['Statut'] == 'SUCCESS') & (dfpmt['CINETPAY'] == 0)]
trx_succes_abs_cinet

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre
0,2024-09-17 00:10:37,NaN,OM_ML,Mel Bet ML,f4c4fd44-1fdb-464a-8f1f-84a84a3edda6,2761322127,EA240917.001045.B275933,NaN,22392707438,NaN,...,NaN,NaN,NaN,NaN,0.00,450.00,0,0,0,15000.00
1,2024-09-17 00:08:32,NaN,TMONEY_TG,1XBET TG,d4a8c7ad-9a10-4759-985e-7bf49c09738e,2761305321,EA240917.000844.S107753,NaN,22871652079,NaN,...,NaN,NaN,NaN,NaN,0.00,850.00,0,0,0,17000.00
2,2024-09-17 00:10:47,NaN,OM_ML,Mel Bet ML,64a5c762-ec80-460c-9407-17f4cdb0606c,2761324891,EA240917.001055.Y724983,NaN,22374251139,NaN,...,NaN,NaN,NaN,NaN,0.00,300.00,0,0,0,10000.00
3,2024-09-17 00:08:46,NaN,TMONEY_TG,1XBET TG,c49fa55f-a12b-4983-b42d-4841042aa3d1,2761321091,EA240917.000858.R712463,NaN,22870018014,NaN,...,NaN,NaN,NaN,NaN,0.00,5045.00,0,0,0,100900.00
4,2024-09-17 00:18:05,NaN,OM_BF,BETMOMO BF,6369ce94-0865-4243-b677-1b979fcb68a3,BWT4004921NTR55E,EA240917.001817.B409445,NaN,22666556297,NaN,...,NaN,NaN,NaN,NaN,0.00,160.00,0,0,0,4000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16479,2024-09-19 02:47:35,NaN,MOOV_BF,Bet Winner BF,1317a099-d121-4bd6-86ba-c844638064e0,138873317,EA240919.024743.E252137,NaN,22653391243,NaN,...,NaN,NaN,NaN,NaN,0.00,55.00,0,0,0,1000.00
16480,2024-09-19 02:47:37,NaN,OM_BF,Bet Winner BF,b77fa4cc-877d-4caa-aafa-28d081789d60,138873346,EA240919.024748.J244234,NaN,22664971079,NaN,...,NaN,NaN,NaN,NaN,0.00,495.00,0,0,0,9000.00
16482,2024-09-19 02:47:41,NaN,OM_BF,1XBET BF,77899fe5-c673-4296-97da-2d64bbacb084,2769433235,EA240919.024750.K632722,NaN,22675788000,NaN,...,NaN,NaN,NaN,NaN,0.00,3360.00,0,0,0,56000.00
16483,2024-09-19 02:48:10,NaN,OM_BF,1XBET BF,1e01d6b8-9876-4305-9295-14b7da1cab27,2769433553,EA240919.024819.W107937,NaN,22674591510,NaN,...,NaN,NaN,NaN,NaN,0.00,2100.00,0,0,0,35000.00


In [30]:
 # Enregistrer les données filtrées dans un fichier Excel

#output_file_path = 'D:\RECO\CINETPAY\PAYOUT\MAJ/trx_a_mettre_ajour.xlsx'
#maj.to_excel(output_file_path, index=False)

#print(f"Fichier enregistré sous : {output_file_path}")


In [31]:
report_date = "20240918"
# Générer automatiquement le chemin de sortie avec la date actuelle
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = 'D:/RECO/CINETPAY/PAYOUT/REACPANALYSE/SEPT/'
output_file_name = f"{current_date}_RAPPORT_ANALYSE_{report_date}.xlsx"
output_file_path = os.path.join(output_dir, output_file_name)
# Chemin du fichier de sortie

# Création de DataFrames d'exemple
data_pmt = dfpmt
data_op = dfop
REC_NEW=new_rec
REJ=rej
pertes = perte
maj_jour_pending_a_success = maj
trxenattente_abs = trx_pending_abs_cinet
trx_succes_abs_cinetpay = trx_succes_abs_cinet
trx_cinetpay_succes_abs_pmt = trx_succes_a_rechercher
# Dictionnaire de DataFrames
feuilles = {
    'DATA PMT': data_pmt,
    'data op': data_op,
    'Rec_New' :REC_NEW,
    'rej' : REJ,
    'pertes': pertes,
    'maj_pending' : maj_jour_pending_a_success,
    'trxenattente_abs': trx_pending_abs_cinet,
    'trx_succes_pmt_abs_cinet' :trx_succes_abs_cinetpay,
    'trx_succes_cinetpay_abs_pmt' :trx_cinetpay_succes_abs_pmt
}

# Création de l'objet ExcelWriter
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for nom_feuille, df in feuilles.items():
        df.to_excel(writer, sheet_name=nom_feuille, index=False)

print(f"Fichier Excel avec plusieurs feuilles sauvegardé avec succès: {output_file_path}")

Fichier Excel avec plusieurs feuilles sauvegardé avec succès: D:/RECO/CINETPAY/PAYOUT/REACPANALYSE/SEPT/20240919_091412_RAPPORT_ANALYSE_20240918.xlsx


In [32]:
select=dfpmt.groupby(['Statut','DateCourte'])['Montant'].count()
select

Statut   DateCourte
FAILED   2024-09-17     781
         2024-09-18     172
         2024-09-19      13
PENDING  2024-09-17       1
         2024-09-18     371
         2024-09-19     200
SUCCESS  2024-09-17    7465
         2024-09-18    5850
         2024-09-19    1632
Name: Montant, dtype: int64

In [33]:
trx = dfpmt.loc[(dfpmt['Statut'] == 'en attente') & (dfpmt['DateCourte'] == '2024-06-10')]
trx

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


ANALYSE PMT

In [34]:
succes= dfpmt[(dfpmt['Statut'] == 'SUCCESS')]
echec =dfpmt[(dfpmt['Statut'] == 'FAILED')]

In [35]:
success=succes[['Created Date',
           'Payment Date',
           'Operator',
           'Merchant Name',
           'Transaction ID',
           'Phone Number',
           'DateCourte',
           'DATEOP',
           'Statut',
           'STATUTOP',
           'OPERATOROP',
           'Montant'
]]
success.head(2)

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,Phone Number,DateCourte,DATEOP,Statut,STATUTOP,OPERATOROP,Montant
0,2024-09-17 00:10:37,NaN,OM_ML,Mel Bet ML,f4c4fd44-1fdb-464a-8f1f-84a84a3edda6,22392707438,2024-09-17,NaN,SUCCESS,NaN,NaN,15000.00
1,2024-09-17 00:08:32,NaN,TMONEY_TG,1XBET TG,d4a8c7ad-9a10-4759-985e-7bf49c09738e,22871652079,2024-09-17,NaN,SUCCESS,NaN,NaN,17000.00


In [36]:
echecs=echec[['Created Date',
           'Payment Date',
           'Operator',
           'Merchant Name',
           'Transaction ID',
           'Phone Number',
           'DateCourte',
           'DATEOP',
           'Statut',
           'STATUTOP',
           'OPERATOROP',
           'Montant'
]]
echecs.head(2)

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,Phone Number,DateCourte,DATEOP,Statut,STATUTOP,OPERATOROP,Montant
27,2024-09-17 00:07:26,NaN,OM_BF,Mel Bet BF,22d985cd-d851-44d9-bb72-d36681415127,22667312708,2024-09-17,NaN,FAILED,NaN,NaN,1000.00
73,2024-09-17 00:23:53,NaN,OM_BF,Mel Bet BF,26a3b041-0705-4d01-b3da-61f0cfea5af7,22656015499,2024-09-17,NaN,FAILED,NaN,NaN,2000.00


In [37]:
select_success_min = success.groupby(['DateCourte','STATUTOP'])['Montant'].min()
select_success_min.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-09-18,VAL,400.00


In [38]:
select_success_max = success.groupby(['DateCourte','STATUTOP'])['Montant'].max()
select_success_max.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-09-18,VAL,1020000.00


In [39]:
select_echec_min = echecs.groupby(['DateCourte', 'STATUTOP'])['Montant'].min()
select_echec_min.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-09-18,REJ,500.00


In [40]:
select_echec_max = echecs.groupby(['DateCourte', 'STATUTOP'])['Montant'].max()
select_echec_max.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-09-18,REJ,200000.00


REQUETE SUPPORT

In [41]:
select_marchand = df_filteredpmt.groupby('Merchant Name').agg(
    Nombre =('Montant', 'count'),
    Volume =('Montant', 'sum')
)
select_marchand

,Nombre,Volume
Merchant Name,,
1XBET BF,976,23393345.00
1XBET CD,165,8503125.00
1XBET TG,1324,13216066.00
7BET ML,1,1000.00
AFROPARI BF,5,45445.00
AFROPARI CI,42,505114.00
AMCDRC LTD CI,320,5709649.00
AVEONET BF,15,215540.00
BETMOMO BF,45,1181406.00


In [42]:
requete_support =dfpmt.loc[(dfpmt['Transaction ID'] =='253f77e6-31d3-45d2-be70-a61ba3f601aa')]
requete_support

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Phone Number,client_email,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre
